<a href="https://colab.research.google.com/github/matsunori39/How-Deep-Learning-AI-Works/blob/main/How_Deep_Learning_AI_Works.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://info.nikkeibp.co.jp/media/NSW/atcl/books/111100050/

# Chapter 7: Making Sense of Image Recognition AI Code

## 7-2 Prepare training and test data

### Import required modules

In [3]:
import torch
import torchvision
import torchvision.transforms as transforms

### Loading training and test data

In [4]:
# image transformation process
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# Load training data
trainset = torchvision.datasets.CIFAR10(root='./data',
                                        train=True,
                                        download=True,
                                        transform=transform)
trainloader = torch.utils.data.DataLoader(trainset,
                                          batch_size=4,
                                          shuffle=True,
                                          num_workers=2)

# Load test data
testset = torchvision.datasets.CIFAR10(root='./data',
                                        train=False,
                                        download=True,
                                        transform=transform)
testloader = torch.utils.data.DataLoader(testset,
                                          batch_size=4,
                                          shuffle=False,
                                          num_workers=2)

# Define class label names
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified
